In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import nquad

from keras.models import load_model

# Load all the models we will use to make predictions
outcome_models = [] # Models for predicting probabilities for Fair Catch, Return, No Field outcomes
field_models = [] # Models for predicting whether a punt is Fielded or not
returnlen_models = [] # Models for predicting the return length of a punt
for i in range(5):
    outcome_models.append(load_model(f'./Models/ThreeOutcomeModel{i}.h5'))
    field_models.append(load_model(f'./Models/FieldingModel{i}.h5'))
    returnlen_models.append(load_model(f'./Models/ReturnModel{i}.h5'))
    
mdn_model = load_model('./Models/MDN_BivariateNorm.h5',compile=False) # Model for finding location of bouncing ball

# Normalization statistics
field_norm_stats = pd.read_csv('../ReducedData/training_statistics.csv').values # Outcome/Field models
return_norm_stats = pd.read_csv('../ReducedData/return_training_statistics.csv').values # Return models
bounce_norm_stats = pd.read_csv('../ReducedData/bounce_training_statistics.csv').values # Return models

data = pd.read_csv('../ReducedData/ExtraPuntLocations.csv') # Untrained punts
trained_data = pd.read_csv('../ReducedData/BC_AllPlayers.csv') # Raw data for the punt training data

In [30]:
fb_land = pd.read_csv('../ReducedData/fb_land.csv')
fb_land.loc[fb_land['playDirection']=='left', 'x'] = 120 - fb_land.loc[fb_land['playDirection']=='left', 'x']
fb_land.loc[fb_land['playDirection']=='left', 'y'] = (160/3 - fb_land.loc[fb_land['playDirection']=='left', 'y']).round(2)

real_land = pd.concat([data.merge(fb_land[['gameId','playId','x','y','hangTime']].rename({'x':'x_land','y':'y_land'},axis=1)), trained_data.iloc[:,:-2]],axis=0).reset_index(drop=True)
real_land = real_land[real_land['gameId']//10**6 == 2020]

In [31]:
# Calculate the magnitude of the velocity of a punt given its distance travelled and hangtime
def calc_const(dx, dy, ht):
    g = 32.1/3 # Gravitational constant in yards/s**2
    dist = np.sqrt(dx**2 + dy**2)
    v_const = (dist/ht)**2 + (g*ht/2)**2
    return np.sqrt(v_const)

# Calculate the hangtime of a punt with a given velocity (magnitude) and the distance travelled
# Return two solutions corresponding to low-kick and high-kick solution
def calc_time(del_x, v_const):
    g = 32.1/3 # Gravitational constant in yards/s**2
    p1 = 2/g**2*v_const**2
    p2 = 2/g**2*np.sqrt(v_const**4 - g**2 * del_x**2)
    return [np.sqrt(p1-p2),np.sqrt(p1+p2)]



# Several Functions that will be useful in converting the raw features of a punt to the specific features of each model


# Calculate the array of coordinate differences between two sets of teams
def diff_coord(team1, team2):
    t1 = team1.stack().reset_index()
    t2 = team2.stack().reset_index()
    t1xt2 = t1.merge(t2,on='level_0')
    t1xt2['diff'] = t1xt2['0_y'] - t1xt2['0_x']
    return t1xt2.set_index(['level_0','level_1_y','level_1_x'])['diff']


# Calculate the array of coordinate differences between a particular team and some coordinate
def diff_team_from_loc(team, coord):
    return -team.sub(coord,axis=0)


# Standardize features given the normalization statistics
def standardize_features(tensor, norm_stats):
    norm_tensor = np.copy(tensor)
    for i in range(tensor.shape[-1]):
        norm_tensor[...,i] = (norm_tensor[...,i] - norm_stats[i,0])/norm_stats[i,1]
    return norm_tensor


# Isolate each teams x and y coordinates
def team_coords(df):
    punt_x = df.iloc[:,5:24:2]
    punt_y = df.iloc[:,6:25:2]
    ret_x = df.iloc[:,25:46:2]
    ret_y = df.iloc[:,26:47:2]
    return [punt_x,punt_y,ret_x,ret_y]


# Convert the raw data into the specific features used in the outcome_models and field_models
# Predict probability of a punt being fielded, fair caught, returned, or not fielded
def convert_to_field_ft(df):
    features = np.zeros((len(df),11,10,9))
    punt_x,punt_y,ret_x,ret_y = team_coords(df)
    
    features[:,:,:,0] = diff_coord(punt_x,ret_x).unstack().values.reshape(len(df),11,10)
    features[:,:,:,1] = diff_coord(punt_y,ret_y).unstack().values.reshape(len(df),11,10)
    
    features[:,:,:,2] = np.tile((diff_team_from_loc(ret_x, df['x_punt']).values)[:,:,np.newaxis],10)
    features[:,:,:,3] = np.tile((diff_team_from_loc(ret_y, df['y_punt']).values)[:,:,np.newaxis],10)
    
    features[:,:,:,4] = np.tile((diff_team_from_loc(ret_x, df['x_land']).values)[:,:,np.newaxis],10)
    features[:,:,:,5] = np.tile((diff_team_from_loc(ret_y, df['y_land']).values)[:,:,np.newaxis],10)
    
    features[:,:,:,6] = np.tile(df['x_fb'].values,(10,11,1)).T
    features[:,:,:,7] = np.tile(df['y_fb'].values,(10,11,1)).T
    
    features[:,:,:,8] = np.tile(df['hangTime'].values,(10,11,1)).T
    
    return standardize_features(features, field_norm_stats)


# Convert the raw data into the specific features used in the returnlen_models
# Length of return predictions
def convert_to_return_ft(df):
    features = np.zeros((len(df),10,11,7))
    punt_x,punt_y,ret_x,ret_y = team_coords(df)
    
    features[:,:,:,0] = diff_coord(ret_x,punt_x).unstack().values.reshape(len(df),10,11)
    features[:,:,:,1] = diff_coord(ret_y,punt_y).unstack().values.reshape(len(df),10,11)
    
    features[:,:,:,2] = np.tile((diff_team_from_loc(ret_x, df['x_punt']).values)[:,np.newaxis],(1,10,1))
    features[:,:,:,3] = np.tile((diff_team_from_loc(ret_y, df['y_punt']).values)[:,np.newaxis],(1,10,1))
    
    features[:,:,:,4] = np.tile((diff_team_from_loc(punt_x, df['x_land']).values)[:,:,np.newaxis],11)
    features[:,:,:,5] = np.tile((diff_team_from_loc(punt_y, df['y_land']).values)[:,:,np.newaxis],11)
    
    features[:,:,:,6] = np.tile(df['hangTime'].values,(11,10,1)).T
    
    return standardize_features(features, return_norm_stats)


# Convert the raw data into the specific features used in the mdn_model
# Bounce location statistic predictions
def convert_to_bounce_ft(df):
    features = np.zeros((len(df),5))
    
    dx = (df['x_land']-df['x_punt']).values
    dy = (df['y_land']-df['y_punt']).values
    v_c = calc_const(dx, dy, df['hangTime'].values)
    dist = np.sqrt(dx**2 + dy**2)
    
    features[:,0] = df['x_fb'].values
    features[:,1] = (df['y_land'] - df['y_punt']).values
    features[:,2] = df['hangTime'].values
    features[:,3] = v_c
    features[:,4] = dist/(v_c*df['hangTime'].values)
    
    return standardize_features(features, bounce_norm_stats)


# Return the three multi-dim arrays to be fed into models for predictions
# outcome_models/field_models , returnlen_models , mdn_model
def get_model_features(df):
    return [convert_to_field_ft(df),convert_to_return_ft(df),convert_to_bounce_ft(df)]

In [32]:
# Convert punts into features and get model predictions for each
field_ft, return_ft, bounce_ft = get_model_features(real_land)

num_models = 5
field_pred = np.zeros((field_ft.shape[0], 4*num_models))
return_pred = np.zeros((return_ft.shape[0], num_models))
bounce_pred = mdn_model.predict(bounce_ft)

In [33]:
for i in range(num_models):
    # Each model in outcome_models will write to the array in the form (P_FairCatch, P_NoField, P_Return)
    # Each model in field_models will write to the array  1-P_Field
    # The field prediction array will have each pair of models recording (P_FC, P_NF, P_R, 1-P_F)
    field_pred[:,4*i:4*i+3] = outcome_models[i].predict(field_ft)
    field_pred[:,4*i+3] = field_models[i].predict(field_ft).reshape(-1,)
    
    # Return length will just record each model's prediction
    return_pred[:,i] = returnlen_models[i].predict(return_ft).reshape(-1,)

    

# For the fielding probabilities, we take an ensemble average over the model predictions
ens_avg_pred = np.zeros((field_pred.shape[0],4))

# Average probability that the punt will not be fielded
ens_avg_pred[:,0] = 1/10*(field_pred[:,1::4].sum(axis=1) + field_pred[:,3::4].sum(axis=1))

# Average probability that the punt will be fielded
# i.e. [0] + [1] = 1.00
ens_avg_pred[:,1] = 1/10*((1-field_pred[:,3::4]).sum(axis=1) + field_pred[:,0::4].sum(axis=1) + field_pred[:,2::4].sum(axis=1))

# Average probability for a Fair Catch (2) and a return (3)
# i.e. [2] + [3] = [1]
ens_avg_pred[:,2] = ens_avg_pred[:,1] * field_pred[:,0::4].sum(axis=1)/(field_pred[:,0::4].sum(axis=1)+field_pred[:,2::4].sum(axis=1))
ens_avg_pred[:,3] = ens_avg_pred[:,1] * field_pred[:,2::4].sum(axis=1)/(field_pred[:,0::4].sum(axis=1)+field_pred[:,2::4].sum(axis=1))

# Combine fielding and return predictions for each punt into a DataFrame
ens_df = pd.concat([real_land[['gameId','playId','x_fb','y_fb','x_land','y_land']].reset_index(drop=True),pd.DataFrame(ens_avg_pred, columns=['NF_avg','Field_avg','FC_avg','R_avg'])],axis=1)
ens_df['R_len'] = np.sinh(return_pred).mean(axis=1)

In [34]:
real_land['dist'] = np.sqrt((real_land['x_land']-real_land['x_punt'])**2 + (real_land['y_land']-real_land['y_punt'])**2)

# Change of basis from x-y coordinates into those defined by the direction of the punt (punter -> landing location)
cos = ((real_land['x_land'] - real_land['x_punt'])/real_land['dist']).values
sin = ((real_land['y_land'] - real_land['y_punt'])/real_land['dist']).values

# Mean predicted bounce length in the punt-direction coordinates
mu_v = bounce_pred[:,0]
mu_vp = bounce_pred[:,1]

# Roughly approximate the expected x,y values of the bounces
ens_df['x_exp_bounce'] = mu_v*cos - mu_vp*sin
ens_df['y_exp_bounce'] = mu_v*sin + mu_v*cos

# Additional statistics about the punt and bounce
ens_df['x_to_ez'] = 110 - ens_df['x_land']
ens_df['y_to_oob'] = (80/3-np.abs(80/3 - ens_df['y_land'])) * np.sign(ens_df['y_land']-80/3)
ens_df['y_rat'] = ens_df['y_to_oob']/ens_df['y_exp_bounce']
ens_df['x_exp_notb'] = ens_df['x_exp_bounce']*ens_df['y_rat']
ens_df.loc[(ens_df['y_rat']>1)|(ens_df['y_rat']<0),'x_exp_notb'] = ens_df['x_exp_bounce']

In [37]:
# Bounce model outputs statistics of the bivariate normal distributions
# Functions to be used in calculating the probabilities and expected values

# Standardize the random variables in the exponential of distributions
def standardize(x,mu,sigma):
    return (x-mu)/sigma

# Given a set of distribution parameters, return the bivariate normal distribution function
def get_bivnorm(mx,my, sx,sy, r):
    def biv_norm(x,y):
        x_stand = standardize(x,mx,sx)
        y_stand = standardize(y,my,sy)
        return 1/(2*np.pi*sx*sy*np.sqrt(1-r**2)) * np.exp(-1/(2*(1-r**2))*(x_stand**2+y_stand**2+2*r*x_stand*y_stand))
    return biv_norm

# Rotate on-field coordinates to punt-direction coordinates before acquiring PDF
def get_rot_pdf(mx,my, sx,sy, r, ctheta,stheta):
    pdf = get_bivnorm(mx,my, sx,sy, r)
    def rotated_biv_norm(x,y):
        x_rot = x*ctheta + y*stheta
        y_rot = y*ctheta - x*stheta
        return pdf(x_rot,y_rot)
    return rotated_biv_norm

# Calculate the expected value of x-field coordinate
def get_rot_ExpX(mx,my, sx,sy, r, ctheta,stheta):
    pdf = get_bivnorm(mx,my, sx,sy, r)
    def rotated_expX(x,y):
        x_rot = x*ctheta + y*stheta
        y_rot = y*ctheta - x*stheta
        return x * pdf(x_rot,y_rot)
    return rotated_expX

# Calculate expected value of x-field coordinate when the ball bounces out of bound
# Assuming a straight path from the bounce location to the final location, the x-value is truncated when the ball leaves play
def get_rot_truncatedExpX(mx,my, sx,sy, r, ctheta,stheta, y_thres):
    pdf = get_bivnorm(mx,my, sx,sy, r)
    def rotated_truncexpX(x,y):
        x_rot = x*ctheta + y*stheta
        y_rot = y*ctheta - x*stheta
        return x*y_thres/y * pdf(x_rot,y_rot)
    return rotated_truncexpX

# Returns y-field integration bounds depending on whether the punt is closer to the y = 0 or y = 160/3 sidelines
def y_oob(y_togo):
    ranges = [[-np.inf,y_togo],[y_togo,np.inf]]
    if y_togo>0:
        return ranges
    return [ranges[1],ranges[0]]

# Return x-field integration bounds, for integrating both touchbacks and out of bounds
def get_XBounds(x_ez,y_oob):
    slope = x_ez/y_oob
    def tb_xbound(y):
        x_minbound = slope*(y-y_oob)+x_ez
        return [x_minbound, np.inf]

    def oob_xbound(y):
        x_upbound = slope*(y-y_oob)+x_ez
        return [x_ez, x_upbound]
    return [tb_xbound,oob_xbound]

In [38]:
for index in ens_df.index.values:
    if index % 100 == 0:
        print(index)
    xl = ens_df.loc[index, 'x_land']
    yl = ens_df.loc[index,'y_land']

    x_to_ez = ens_df.loc[index,'x_to_ez']
    y_to_oob = ens_df.loc[index, 'y_to_oob']
    y_bounds = y_oob(y_to_oob)
    x_bound_tb, x_bound_oob = get_XBounds(x_to_ez, y_to_oob)

    rotated_PDF = get_rot_pdf(*bounce_pred[index], cos[index], sin[index])
    rotated_ExpX = get_rot_ExpX(*bounce_pred[index], cos[index], sin[index])
    rot_trunc_ExpX = get_rot_truncatedExpX(*bounce_pred[index], cos[index], sin[index],y_to_oob)

    # Evaluate the integrals
    # Note, it is faster to split the conditional integration bounds due to the decrease in function calls
    ex_ip = nquad(rotated_ExpX, [[-110,x_to_ez],y_bounds[0]])[0]
    p_tb = nquad(rotated_PDF, [[x_to_ez,np.inf],y_bounds[0]])[0] + nquad(rotated_PDF, [x_bound_tb,y_bounds[1]])[0]
    ex_oob = nquad(rot_trunc_ExpX,[[-110,x_to_ez],y_bounds[1]])[0] + nquad(rot_trunc_ExpX,[x_bound_oob,y_bounds[1]])[0]
    
    ens_df.loc[index, 'p_tb'] = p_tb
    ens_df.loc[index, 'ex_ntb'] = ex_ip + ex_oob

0


C:\Users\rxsim\anaconda3\envs\NFLBigData\lib\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


In [41]:
ens_df.to_csv('../PredictedData/RealLocationValues.csv',index=False)